## Bài 3: GA

### Cài đặt các phương pháp chọn lọc, lai ghép và đột biến cho bài toán Max One Problem

In [4]:
import numpy as np
import random
import math


def fitness_binary(chrom):
    return chrom.count("1")

def tournament_selection(population, fitness_func, k=3):
    group = random.sample(population, k)
    return max(group, key=fitness_func)

def roulette_selection(population, fitness_func):
    fitnesses = [fitness_func(pop) for pop in population]
    total_fit = sum(fitnesses)
    probs = [f/total_fit for f in fitnesses]
    cum_probs = [sum(probs[:i+1]) for i in range(len(probs))]
    r = random.random()
    for i, cp in enumerate(cum_probs):
        if r <= cp:
            return population[i]

def rank_selection(population, fitness_func):
    sorted_pop = sorted(population, key=fitness_func, reverse=True)
    n = len(sorted_pop)
    probs = [(n-i) / (n*(n+1)/2) for i in range(n)]
    cum_probs = [sum(probs[:i+1]) for i in range(n)]
    r = random.random()
    for i, cp in enumerate(cum_probs):
        if r <= cp:
            return sorted_pop[i]

def single_point_crossover(p1, p2):
    point = random.randint(1, len(p1) - 1)
    child1 = p1[:point] + p2[point:]
    child2 = p2[:point] + p1[point:]
    return child1, child2

def two_point_crossover(p1, p2):
    point1, point2 = sorted(random.sample(range(len(p1)), 2))
    child1 = p1[:point1] + p2[point1:point2] + p1[point2:]
    child2 = p2[:point1] + p1[point1:point2] + p2[point2:]
    return child1, child2 


def uniform_crossover(p1, p2):
    c1, c2 = "", ""
    for i in range(len(p1)):
        if random.random() < 0.5:
            c1 += p1[i]
            c2 += p2[i]
        else:
            c1 += p2[i]
            c2 += p1[i]
    return c1, c2

def binary_mutation(chrom):
    pos = random.randint(0, len(chrom)-1)
    chrom = list(chrom)
    chrom[pos] = '1' if chrom[pos] == '0' else '0'
    return ''.join(chrom)

In [7]:
if __name__ == "__main__":
    population = ["101010", "111000", "110110", "000111", "111100", "101111"]
    print("Population:", population)
    print("Fitness:", [fitness_binary(ind) for ind in population])

    parent1_ro = roulette_selection(population, fitness_binary)
    parent2_ro = roulette_selection(population, fitness_binary)
    parent1_ra = rank_selection(population, fitness_binary)
    parent2_ra = rank_selection(population, fitness_binary)
    parent1_to = tournament_selection(population, fitness_binary)
    parent2_to = tournament_selection(population, fitness_binary)
    print("\nRoulette:", parent1_ro, parent2_ro)
    print("Rank:", parent1_ra, parent2_ra)
    print("Tournament:", parent1_to, parent2_to)

    c1, c2 = single_point_crossover(parent1_to, parent2_to)
    print("\nSingle-point crossover:", c1, c2)

    c3, c4 = two_point_crossover(parent1_to, parent2_to)
    print("Two-point crossover:", c3, c4)

    c5, c6 = uniform_crossover(parent1_to, parent2_to)
    print("Uniform crossover:", c5, c6)

    print("\nMutation of", c1, "->", binary_mutation(c1))
    print("\nMutation of", c2, "->", binary_mutation(c2))
    print("\nMutation of", c3, "->", binary_mutation(c3))
    print("\nMutation of", c4, "->", binary_mutation(c4))
    print("\nMutation of", c5, "->", binary_mutation(c5))
    print("\nMutation of", c6, "->", binary_mutation(c6))

Population: ['101010', '111000', '110110', '000111', '111100', '101111']
Fitness: [3, 3, 4, 3, 4, 5]

Roulette: 000111 111100
Rank: 111000 111100
Tournament: 101111 101111

Single-point crossover: 101111 101111
Two-point crossover: 101111 101111
Uniform crossover: 101111 101111

Mutation of 101111 -> 101101

Mutation of 101111 -> 101101

Mutation of 101111 -> 001111

Mutation of 101111 -> 100111

Mutation of 101111 -> 111111

Mutation of 101111 -> 001111


### GA cho bài toán Sphere

In [8]:
import numpy as np
def sphere(x):
    return np.sum(x ** 2)

def initialize_pop(pop_size=20, dim=5, bounds=(-5,5)):
    return np.random.uniform(bounds[0], bounds[1], (pop_size, dim))

def fitness(ind):
    return -sphere(ind)


def tournament_selection(pops, k=3):
    idxs = np.random.choice(len(pops), k, replace=False)
    winner = max(idxs, key=lambda idx: fitness(pops[idx]))
    return pops[winner].copy()


def crossover(p1, p2, crossover_rate=0.9):
    if np.random.rand() < crossover_rate:
        point = np.random.randint(1, len(p1))
        c1 = np.concatenate([p1[:point], p2[point:]])
        c2 = np.concatenate([p2[:point], p1[point:]])
        return c1, c2
    return p1.copy(), p2.copy()


def mutate(ind, mutation_rate=0.1, bounds=(-5, 5)):
    for i in range(len(ind)):
        if np.random.rand() < mutation_rate:
            ind[i] += np.random.normal(0, 0.1)
            ind[i] = np.clip(ind[i], bounds[0], bounds[1])
    return ind

def genetic_algorithm(generations=50, pop_size=20, dim=5):
    pops = initialize_pop(pop_size, dim)
    best_fit = float("inf")
    best_sol = None
    for g in range(generations):
        new_pop = []
        while len(new_pop) < pop_size:
            p1 = tournament_selection(pops)
            p2 = tournament_selection(pops)
            c1, c2 = crossover(p1, p2)
            c1 = mutate(c1)
            c2 = mutate(c2)
            new_pop.extend([c1, c2])
        pops = np.array(new_pop[:pop_size])
        fits = [sphere(ind) for ind in pops]
        gen_best = min(fits)
        if gen_best < best_fit:
            best_fit = gen_best
            best_sol = pops[np.argmin(fits)]
        print(f"Gen {g+1}: best = {best_fit:.6f}")
    return best_sol, best_fit

In [9]:
best_sol, best_fit = genetic_algorithm()
print("\nBest solution:", best_sol)
print("Best fitness (Sphere):", best_fit)

Gen 1: best = 10.246485
Gen 2: best = 10.246485
Gen 3: best = 9.605778
Gen 4: best = 4.951483
Gen 5: best = 3.952521
Gen 6: best = 3.386376
Gen 7: best = 3.139956
Gen 8: best = 3.139956
Gen 9: best = 2.882720
Gen 10: best = 2.877066
Gen 11: best = 2.664080
Gen 12: best = 2.295926
Gen 13: best = 1.864946
Gen 14: best = 1.864946
Gen 15: best = 1.784227
Gen 16: best = 1.757215
Gen 17: best = 1.719658
Gen 18: best = 1.500721
Gen 19: best = 1.500721
Gen 20: best = 1.385337
Gen 21: best = 1.385337
Gen 22: best = 1.328023
Gen 23: best = 1.075155
Gen 24: best = 1.075155
Gen 25: best = 1.075155
Gen 26: best = 1.075155
Gen 27: best = 1.075155
Gen 28: best = 1.036156
Gen 29: best = 1.020168
Gen 30: best = 0.960434
Gen 31: best = 0.926115
Gen 32: best = 0.926115
Gen 33: best = 0.736566
Gen 34: best = 0.736566
Gen 35: best = 0.695581
Gen 36: best = 0.576001
Gen 37: best = 0.556944
Gen 38: best = 0.527096
Gen 39: best = 0.527096
Gen 40: best = 0.527023
Gen 41: best = 0.486949
Gen 42: best = 0.266907